<img src="https://storage.googleapis.com/gweb-uniblog-publish-prod/images/gemma-header.width-1200.format-webp.webp" width="100%">

## Instruct Fine-tuning [Gemma](https://blog.google/technology/developers/gemma-open-models/) using qLora and Supervise Finetuning

This is a comprahensive notebook and tutorial on how to fine tune the `gemma-2b-it` Model

All the code will be available on my Github. Do drop by and give a follow and a star⭐.\
[sagarvk24](https://github.com/sagarvk24)
\
[Github Code](https://github.com/sagarvk24/Fine-Tune-LLMs-.git)

Check for GPU (In my case, using Kaggle's T4 x 2)

In [1]:
!nvidia-smi

Tue Feb  4 20:12:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Step 1: Install the Dependencies

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0 #To load the quantized version of the model
!pip3 install -q -U peft==0.8.2 #For Parameter-Efficient Fine Tuning
!pip3 install -q -U trl==0.7.10 #Supervised Fine Tuning 
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0 #for loading the datasets
!pip3 install -q -U transformers==4.38.0 #for main processing and training 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.
s3fs 2024.9.0 requires fsspec==2024.9.0.*, but you have fsspec 2023.10.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━

We will load the model using QLoRA Quantization to reduce the usage of memory, as it is very difficult to load such big LLM and fine tune. 

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #Quantization
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16 #To avoid information loss while fine tuning
)

# Step 2: Hugging Face: Log In and Access Key Setup 

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#In case using Google Colab
#Paste the access token in Colab's Environment 

#import os
#from google.colab import userdata
#os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Step 3: Load the Model

In [5]:
#Difference between base model and 'it' model is that, base model is the most raw form of the LLM model. The output is produced in a very raw format, whereas 'it' model produces outputs in a very 

# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model
  

  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

### Testing 

In [8]:
result = get_completion(query="code to check a number is even or odd, in C++", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code to check a number is even or odd, in C++
  
model
  

    ```c++
  #include <iostream>
  #include <cmath>

  using namespace std;

  int main() {
    int number;

    // Get the number from the user
    cout << "Enter a number: ";
    cin >> number;

    // Check if the number is even
    if (number % 2 == 0) {
      cout << "The number " << number << " is even" << endl;
    } else {
      cout << "The number " << number << " is odd" << endl;
    }

    return 0;
  }
  ```


# Step 4: Load the Dataset

In [11]:
from datasets import load_dataset

# disease_symptoms_path = kagglehub.dataset_download('sagarvk18/disease-symptoms')
dataset = load_dataset(r"/kaggle/input/disease-symptoms", split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 349
})

In [12]:
df = dataset.to_pandas()
df.head(10)

,instruction,input,output
0,Diagnose the patient based on the provided sym...,Patient has Influenza with symptoms: Fever (Ye...,Positive
1,Diagnose the patient based on the provided sym...,Patient has Common Cold with symptoms: Fever (...,Negative
2,Diagnose the patient based on the provided sym...,"Patient has Eczema with symptoms: Fever (No), ...",Negative
3,Diagnose the patient based on the provided sym...,"Patient has Asthma with symptoms: Fever (Yes),...",Positive
4,Diagnose the patient based on the provided sym...,"Patient has Asthma with symptoms: Fever (Yes),...",Positive
5,Diagnose the patient based on the provided sym...,"Patient has Eczema with symptoms: Fever (Yes),...",Positive
6,Diagnose the patient based on the provided sym...,Patient has Influenza with symptoms: Fever (Ye...,Positive
7,Diagnose the patient based on the provided sym...,Patient has Influenza with symptoms: Fever (Ye...,Positive
8,Diagnose the patient based on the provided sym...,Patient has Hyperthyroidism with symptoms: Fev...,Negative
9,Diagnose the patient based on the provided sym...,Patient has Hyperthyroidism with symptoms: Fev...,Negative


# Step 5: Format the Dataset

In [13]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

## Step 5.1: Tokenize the data to make it easier for model to understand

In [14]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

## Step 5.2: Split the Dataset

In [15]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [16]:
print(test_data)

Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 70
})


# Step 6: Apply LoRA (Low Rank Adaptation)
Magic with peft! 
-> Load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [17]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [18]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [19]:
#to determine all the linear layerss while using bitsandbytes
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [20]:
modules = find_all_linear_names(model)
print(modules)

['up_proj', 'down_proj', 'o_proj', 'v_proj', 'gate_proj', 'q_proj', 'k_proj']


In [21]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32, #Adjust according to your resources
    lora_alpha=16,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [22]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 39223296 | total: 2545395712 | Percentage: 1.5410%


# Step 7: Setup for the Training

## Fine-Tuning with QLoRA and Supervised Fine-Tuning using SFTT Trainer

In [30]:
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for padding side warning
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,  # Increased Batch size
        gradient_accumulation_steps=2,  # Adjusted accumulation
        warmup_ratio=0.03,  
        max_steps=20, #Adjust accordingly
        learning_rate=1e-4,  # Stable LR
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_bnb_8bit",  # Efficient optimizer
        save_strategy="epoch",
        fp16=True,  # Enable mixed precision for speedup
        report_to = "none" #can change it to "wandb" to log results there
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [24]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return 2 (since you have T4 x2)

True
2


In [25]:
torch.cuda.empty_cache()

# Step 8: Start the Training!😇❤️

In [26]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [27]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

When working on Kaggle, in case you wish to start the training again, with changed parameters, then run the command first (added just below)

In [29]:
rm -rf outputs/checkpoint-10

In [31]:
model.config.use_cache = False  # Disable cache for training
trainer.train()  # Start fine-tuning

Step,Training Loss
1,3.550000
2,3.505300
3,2.878700
4,2.451800
5,2.103200
6,1.892700
7,1.630600
8,1.474900
9,1.348600
10,1.264700


TrainOutput(global_step=20, training_loss=1.6084246426820754, metrics={'train_runtime': 166.0634, 'train_samples_per_second': 0.963, 'train_steps_per_second': 0.12, 'total_flos': 176806502645760.0, 'train_loss': 1.6084246426820754, 'epoch': 0.57})

# Step 9: Share the Adapters on the 🤗 Hub! 

In [32]:
new_model = "gemma-2B-Instruct-Finetune-MedicalSymptoms-Sagar" #Name of the model you will be pushing to huggingface model hub

In [33]:
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [34]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sagarvk24/gemma-2B-Instruct-Finetune-MedicalSymptoms-Sagar/commit/dc32b0ef59d072ec108911e1f118684481eb7edd', commit_message='Upload tokenizer', commit_description='', oid='dc32b0ef59d072ec108911e1f118684481eb7edd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sagarvk24/gemma-2B-Instruct-Finetune-MedicalSymptoms-Sagar', endpoint='https://huggingface.co', repo_type='model', repo_id='sagarvk24/gemma-2B-Instruct-Finetune-MedicalSymptoms-Sagar'), pr_revision=None, pr_num=None)

### To avoid Warnings

In [38]:
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

# Step 10: Test on Fine-Tuned Model🔎

In [37]:
query = """
Diagnose the patient based on the provided symptoms and information.

Patient has persistent cough and fever with symptoms: Fever (Yes), Cough (Yes), Fatigue (Moderate), Difficulty Breathing (No). 
Age: 35, Gender: Male, Blood Pressure: Normal, Cholesterol Level: High.
"""

result = get_completion(query, model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  
Diagnose the patient based on the provided symptoms and information.

Patient has persistent cough and fever with symptoms: Fever (Yes), Cough (Yes), Fatigue (Moderate), Difficulty Breathing (No). 
Age: 35, Gender: Male, Blood Pressure: Normal, Cholesterol Level: High.

  
model
  

   Analyzing the patient's symptoms and information, here's the preliminary diagnosis:

  Cough with fever may be associated with various conditions. Common causes include respiratory infections such as pneumonia, bronchitis, and influenza. However, the absence of cough in this case is significant and can suggest other possibilities.

   Fever with fatigue is a typical symptom of conditions like viral fevers and infections. High blood pressure and cholesterol levels might also contribute to the patient's symptoms.

  Therefore, further investigations such as a chest X-ray or electrocardiogr

In [39]:
query = """
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Diagnose the patient based on the provided symptoms and information.

Important: Carefully consider the presence or absence of each symptom in the diagnosis.

Patient has persistent cough and fever with symptoms: Fever (Yes), Cough (Yes), Fatigue (Moderate), Difficulty Breathing (No).
Age: 35, Gender: Male, Blood Pressure: Normal, Cholesterol Level: High.
"""

result = get_completion(query=query, model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Diagnose the patient based on the provided symptoms and information.

Important: Carefully consider the presence or absence of each symptom in the diagnosis.

Patient has persistent cough and fever with symptoms: Fever (Yes), Cough (Yes), Fatigue (Moderate), Difficulty Breathing (No).
Age: 35, Gender: Male, Blood Pressure: Normal, Cholesterol Level: High.

  
model
  

   **Possible Diagnosis:** Asthma

  Based on the patient's symptoms, particularly the absence of difficulty breathing, the diagnosis of Asthma is most likely. Asthma is characterized by symptoms such as persistent coughing, fever, and difficulty breathing. However, without the presence of difficulty breathing, the diagnosis of asthma may be less definite. 

It is important to note that this is only